In [1]:
### 1. 특정 경로를 입력받으면 그 경로 안에 있는 xml파일을 다 읽고 
### 아래 함수를 사용해 수정하고 새롭게 저장해서 하위 디렉토리의  폴더에 저장

import os
import xml.etree.ElementTree as ET

In [2]:
## root 를 통해 parsing한 xml을 다시 xml화 시키기 위한 함수
def indent(elem, level=0):
    i = "\n" + level*"  "
    j = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        
        for subelem in elem:
            indent(subelem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = j
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = j
    
    return elem

In [3]:
#path 부분을 Update 하고 새 이름으로 저장하는 함수 [annotation 파일 내의 filename과 이미지파일의 이름이 같아야함.]
def modifiy_annotation(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")
    folder = root.find("folder")
    width=root.find("width")
    height=root.find("height")
    folder.text = 'dataset'
    

    data_type = filename.text[-4:]
    image_file_name = xmlName[0:-4]
    new_filename = image_file_name+data_type
    filename.text = new_filename
    # .text는 안의 값을 불러오는 함수이다.
    #이미지가 저장되어있는 path를 저장 
    image_path_dir = darkflow_path + "/data/dataset/"
    new_path = image_path_dir + str(new_filename) 
    path.text = str(new_path) #new_path를 통해 path 의 값을 update한다.
        
    print(new_path)    
#     try:
#         with open(new_path,'r') as f:
#             pass
#     except:
#         print(filename.text)
    tree =ET.ElementTree(indent(root))
    newXmlName = xmlName
    tree.write(newXmlName, xml_declaration=True, encoding='utf-8')

In [4]:
def find_zero(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")
    folder = root.find("folder")
    size = root.find('size')
    width=size.find("width")
    height=size.find("height")
    folder.text = 'dataset'
    
    if width.text == '0' or height.text =='0':
        os.remove(darkflow_path+"/data/finalXML/"+xmlName)

In [5]:
def check_xml_error(root, darkflow_path, xmlName):
    # filename과 path tag를 find함수로 찾는다.<filename> </filename>  // <path> </path>
    filename = root.find("filename")
    path = root.find("path")
    folder = root.find("folder")
    size = root.find('size')
    width = size.find('width')
    height = size.find('height')
    depth = size.find('depth')
    s = [width.text, height.text, depth.text]
    obj = root.findall('object')
    
    
    
    return filename.text, path.text, folder.text, s, obj

In [6]:
# fname = 'C:\darkflow\data\dataset\apple-0.jpg'
# ET.parse(fname).getroot()

In [7]:
#이 jupyter notebook은 darkflow 폴더 안에 넣고 실행해야함. 그래야 darkflow_path를 os.getcwd()로 잘 받아옴.
darkflow_path = os.getcwd()
darkflow_path

'C:\\darkflow'

In [8]:
#Annotation을 저장된 위치의 path 나같은 경우엔 darkflow_path +/dataset/dataset_0820/ 이라는 폴더 안에 annotation을 다 저장해두었음
anno_dir=darkflow_path+"/data/XML/"
anno_dir

'C:\\darkflow/data/XML/'

In [9]:
#tree.write 함수가 현재 폴더에 annotation파일을 저장하기에 현재 쥬피터의 위치를 변경해야함. 새로 저장할 annotation폴더(나 같은 경우 newdataset)을 만들고 저장해야함.
new_anno_dir = darkflow_path+"/data/finalXML"
os.chdir(new_anno_dir)
os.getcwd()

'C:\\darkflow\\data\\finalXML'

In [10]:
from tqdm import tqdm

In [ ]:
labels = '''
bean_sprout
black_mushroom
bracken
celery
chinese_cabbage
chinese_leek
cucumber
cutlassfish
daikon
kimchi
king_oyster_mushroom
lettuce
lettus
octopus
onion
oyster_mushroom
perilla_leaf
pork
squid
sweet_potato
sweetpumpkin
tofu
tuna_can
udon
water_parsley
'''

In [ ]:
labels = labels.split('\n')[1:-1]
labels

In [13]:
# # os.walk로 anno_dir 안에 있는 파일 이름들을 files에 저장.
# # 그 후 ET.parse와 modifiy_annotation을 통해 path업데이트 후 new_anno_dir에 update된 데이터 저장

remove_list = list()
for root, dirs, files in os.walk(anno_dir):
    for fname in tqdm(files):
        tree = ET.parse(anno_dir+'/'+fname)
        root2 = tree.getroot()
        xmlName = fname
        modifiy_annotation(root2, darkflow_path, xmlName)
#         find_zero(root2, darkflow_path, xmlName)
        file_name, path_name, folder_name, size, objs = check_xml_error(root2, darkflow_path, xmlName)
        
#         for name in [file_name, path_name, folder_name]:
#             if ' ' in name or '\t' in name:
#                 print(xmlName)
                
#         try:
#             with open(path_name,'r') as f:
#                 pass
#         except:
#             print(file_name) 
        
#         if '0' in size:
#             remove_list.append(file_name)
#             print(xmlName,end=' ')
#             os.remove(anno_dir+'/'+fname)
#             print('-> DELETED!!')
            
#         for e in size:
#             if ' ' in e:
#                 print(xmlName)
                
#         if not objs:
#             print('bounding box가 없음 : ',xmlName)
#             print(xmlName,end=' ')
#             os.remove(anno_dir+'/'+fname)
#             print('-> DELETED!!')
            
#         for obj in objs:
#             if ' ' in obj.find('name').text:
#                 print(xmlName)

#             if not(obj.find('name').text in labels):
#                 print(xmlName)
            
#             bndbox = obj.find('bndbox')
            
#             xmin = bndbox.find('xmin').text
#             ymin = bndbox.find('ymin').text
#             xmax = bndbox.find('xmax').text
#             ymax = bndbox.find('ymax').text

#             if '0' in [xmin,ymin,xmax,ymax]:
#                 print(xmlName)
                
            
        
# # remove_list

  0%|                                                                                         | 0/5286 [00:00<?, ?it/s]

C:\darkflow/data/dataset/bean_sprout_2_00010001
C:\darkflow/data/dataset/bean_sprout_2_00040004
C:\darkflow/data/dataset/bean_sprout_2_00080008
C:\darkflow/data/dataset/bean_sprout_2_00150015
C:\darkflow/data/dataset/bean_sprout_2_00200020
C:\darkflow/data/dataset/bean_sprout_2_00230023
C:\darkflow/data/dataset/bean_sprout_2_00290029
C:\darkflow/data/dataset/bean_sprout_2_00380038
C:\darkflow/data/dataset/bean_sprout_2_00410041
C:\darkflow/data/dataset/bean_sprout_2_00430043
C:\darkflow/data/dataset/bean_sprout_2_00450045
C:\darkflow/data/dataset/bean_sprout_2_00460046
C:\darkflow/data/dataset/bean_sprout_2_00470047
C:\darkflow/data/dataset/bean_sprout_2_00500050
C:\darkflow/data/dataset/bean_sprout_2_00540054
C:\darkflow/data/dataset/bean_sprout_2_00560056
C:\darkflow/data/dataset/bean_sprout_2_00580058
C:\darkflow/data/dataset/bean_sprout_2_00600060
C:\darkflow/data/dataset/bean_sprout_2_00630063
C:\darkflow/data/dataset/bean_sprout_2_00640064
C:\darkflow/data/dataset/bean_sprout_2_0

  1%|▍                                                                              | 27/5286 [00:00<00:20, 262.84it/s]

C:\darkflow/data/dataset/bean_sprout_2_00820082
C:\darkflow/data/dataset/bean_sprout_2_00830083
C:\darkflow/data/dataset/bean_sprout_2_00860086
C:\darkflow/data/dataset/bean_sprout_2_00870087
C:\darkflow/data/dataset/bean_sprout_2_00880088
C:\darkflow/data/dataset/bean_sprout_2_00900090
C:\darkflow/data/dataset/bean_sprout_2_00980098
C:\darkflow/data/dataset/bean_sprout_2_00990099
C:\darkflow/data/dataset/bean_sprout_2_01140114
C:\darkflow/data/dataset/bean_sprout_2_01180118
C:\darkflow/data/dataset/bean_sprout_2_01200120
C:\darkflow/data/dataset/bean_sprout_2_01300130
C:\darkflow/data/dataset/bean_sprout_2_01310131
C:\darkflow/data/dataset/bean_sprout_2_01330133
C:\darkflow/data/dataset/bean_sprout_2_01350135
C:\darkflow/data/dataset/bean_sprout_2_01370137
C:\darkflow/data/dataset/bean_sprout_2_01390139
C:\darkflow/data/dataset/bean_sprout_2_01400140
C:\darkflow/data/dataset/bean_sprout_2_01420142
C:\darkflow/data/dataset/bean_sprout_2_01520152
C:\darkflow/data/dataset/bean_sprout_2_0

  1%|▊                                                                              | 52/5286 [00:00<00:20, 255.91it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████▍| 5251/5286 [00:22<00:00, 245.11it/s]

C:\darkflow/data/dataset/water_parsley_600_600
C:\darkflow/data/dataset/water_parsley_601_601
C:\darkflow/data/dataset/water_parsley_602_602
C:\darkflow/data/dataset/water_parsley_603_603
C:\darkflow/data/dataset/water_parsley_604_604
C:\darkflow/data/dataset/water_parsley_605_605
C:\darkflow/data/dataset/water_parsley_606_606
C:\darkflow/data/dataset/water_parsley_607_607
C:\darkflow/data/dataset/water_parsley_608_608
C:\darkflow/data/dataset/water_parsley_609_609
C:\darkflow/data/dataset/water_parsley_610_610
C:\darkflow/data/dataset/water_parsley_611_611
C:\darkflow/data/dataset/water_parsley_612_612
C:\darkflow/data/dataset/water_parsley_613_613
C:\darkflow/data/dataset/water_parsley_614_614
C:\darkflow/data/dataset/water_parsley_615_615
C:\darkflow/data/dataset/water_parsley_616_616
C:\darkflow/data/dataset/water_parsley_617_617
C:\darkflow/data/dataset/water_parsley_618_618
C:\darkflow/data/dataset/water_parsley_619_619
C:\darkflow/data/dataset/water_parsley_620_620
C:\darkflow/d

100%|████████████████████████████████████████████████████████████████████████████▊| 5276/5286 [00:22<00:00, 243.89it/s]

C:\darkflow/data/dataset/water_parsley_625_625
C:\darkflow/data/dataset/water_parsley_626_626
C:\darkflow/data/dataset/water_parsley_627_627
C:\darkflow/data/dataset/water_parsley_628_628
C:\darkflow/data/dataset/water_parsley_629_629
C:\darkflow/data/dataset/water_parsley_630_630
C:\darkflow/data/dataset/water_parsley_631_631
C:\darkflow/data/dataset/water_parsley_632_632
C:\darkflow/data/dataset/water_parsley_633_633
C:\darkflow/data/dataset/water_parsley_634_634


100%|█████████████████████████████████████████████████████████████████████████████| 5286/5286 [00:22<00:00, 238.16it/s]


In [14]:
filepath = darkflow_path+"/data/dataset/"
for file in os.listdir(filepath):
    try:
        with open(filepath+file,'r') as f:
            pass
    except:
        print(file)